###address standardize:

In [ ]:
%%writefile script.sh

#!/bin/bash

input=$1
output=$2
# Columns to capitalize (0-based indices, e.g., 2 for "city" and 3 for "country")
column_to_capitalize=$3

# Process the file
awk -F"," -v col=$column_to_capitalize 'BEGIN {OFS = FS}
NR == 1 {print; next} {
    $col = toupper($col);
    print
}' "$input" > "$output"


Overwriting script.sh


In [ ]:
!chmod +x script.sh

In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Eligible,320,80000,37,1,,PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Eligible,150,35000,48,2,,Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligible,200,45000,25,3,,Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Eligible,140,42000,38,4,,Snohomish County PUD,3.50



Overwriting test_input.csv


In [ ]:
!./script.sh test_input.csv test_output.csv 3

In [ ]:
!cat test_output.csv



VIN,County,CITY,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,SEATTLE,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Eligible,320,80000,37,1,,PSE,3.14
5678,King,REDMOND,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Eligible,150,35000,48,2,,Seattle City Light,3.22
9101,Pierce,TACOMA,WA,98402,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligible,200,45000,25,3,,Tacoma Power,3.10
1122,Snohomish,EVERETT,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Eligible,140,42000,38,4,,Snohomish County PUD,3.50
,,


###Cafv_standardize.sh

In [ ]:
%%writefile replace_eligibility.sh

#!/bin/bash

input=$1
output=$2

# Replace "Clean Alternative Fuel Vehicle Eligible" with "ELIGIBLE" and
# "Not eligible due to low battery range" with "INELIGIBLE" in column 10
# "Eligibility unknown as battery range has not been researched" with "N/A"
awk -F',' -v OFS=',' '
    {
        # Replace in column 10
        if (NR == 1) {
           print;
           next
        }
        if ($10 == "Clean Alternative Fuel Vehicle Eligible") {
            $10 = "ELIGIBLE"
        } else if ($10 == "Not eligible due to low battery range") {
            $10 = "INELIGIBLE"
        } else {
            $10 = "N/A"
        }
        print $0
    }' "$input" > "$output"


Overwriting replace_eligibility.sh


In [ ]:
!chmod +x replace_eligibility.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,,PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,,Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,,Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,,Snohomish County PUD,3.50


Overwriting test_input.csv


In [ ]:
!./replace_eligibility.sh test_input.csv test_output.csv


In [ ]:
!cat test_output.csv



VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,N/A,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),ELIGIBLE,320,80000,37,1,,PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),INELIGIBLE,150,35000,48,2,,Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),N/A,200,45000,25,3,,Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),ELIGIBLE,140,42000,38,4,,Snohomish County PUD,3.50


###Clean_non_WA.sh

In [ ]:
%%writefile filter_wa_rows.sh

#!/bin/bash

input=$1
output=$2

# Use awk to keep the header and only rows where
# the 4th column contains "WA"
awk -F',' -v OFS=',' '
    NR == 1 {print $0}  # Print header (first row)
    # Print rows where the 4th column is "WA"
    NR > 1 && $4 == "WA" {print $0}
' "$input" > "$output"


Overwriting filter_wa_rows.sh


In [ ]:
!chmod +x filter_wa_rows.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,,PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,,Seattle City Light,3.22
9101,Pierce,Tacoma,CA,90210,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,,Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,,Snohomish County PUD,3.50


Overwriting test_input.csv


In [ ]:
!./filter_wa_rows.sh test_input.csv test_output.csv


In [ ]:
!cat test_output.csv



1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,,PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,,Seattle City Light,3.22
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,,Snohomish County PUD,3.50


###Coords_stndardize.sh

In [ ]:
%%writefile remove_point_coordinates.sh

#!/bin/bash

# Input CSV file
input_file=$1
output_file=$2

# Use awk to process each line of the CSV
awk -F, '{
    # Remove POINT ( and ) from column 15
    gsub(/POINT \(|\)/, "", $15)
    print
}' OFS=, "$input_file" > "$output_file"


Overwriting remove_point_coordinates.sh


In [ ]:
!chmod +x remove_point_coordinates.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,Tacoma,CA,90210,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


Overwriting test_input.csv


In [ ]:
!./remove_point_coordinates.sh test_input.csv test_output.csv


In [ ]:
!cat test_output.csv



VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,122.33 -47.61,PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,122.12 -47.67,Seattle City Light,3.22
9101,Pierce,Tacoma,CA,90210,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,122.44 -47.25,Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,122.20 -47.98,Snohomish County PUD,3.50


###Empty_Values

In [ ]:
%%writefile find_empty_rows.sh

#!/bin/bash

input=$1

awk -F',' '{
    for (i = 1; i <= NF; i++) {
        if ($i == "") {
            print $0
            break
        }
    }
}' "$input"


Overwriting find_empty_rows.sh


In [ ]:
!chmod +x find_empty_rows.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,,PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,,Seattle City Light,3.22
9101,Pierce,,CA,90210,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,,Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,,Snohomish County PUD,3.50


Overwriting test_input.csv


In [ ]:
!./find_empty_rows.sh test_input.csv > test_output.csv


In [ ]:
!cat test_output.csv


1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,,PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,,Seattle City Light,3.22
9101,Pierce,,CA,90210,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,,Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,,Snohomish County PUD,3.50


###eu_standardize.sh

In [ ]:
%%writefile remove_inc.sh

#!/bin/bash

input=$1  # Input file
output=$2  # Output file

# Use sed to replace ", INC" with "" in every line
sed 's/, INC//g' "$input" > "$output"


Overwriting remove_inc.sh


In [ ]:
!chmod +x remove_inc.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE, INC,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light, INC,3.22
9101,Pierce,Tacoma,CA,90210,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,POINT (122.44 -47.25),Tacoma Power, INC,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,POINT (122.20 -47.98),Snohomish County PUD, INC,3.50


Overwriting test_input.csv


In [ ]:
!./remove_inc.sh test_input.csv test_output.csv


In [ ]:
!cat test_output.csv



VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,Tacoma,CA,90210,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


###ev_type_standardize.sh

In [ ]:
%%writefile replace_vehicle_type.sh

#!/bin/bash

input=$1
output=$2

# Replace any occurrence of "Plug-in Hybrid Electric Vehicle (PHEV)" with "HYBRID"
# and any occurrence of "Battery Electric Vehicle (BEV)" with "BATTERY" in column 9
awk -F',' -v OFS=',' '
    {
        if ($9 == "Plug-in Hybrid Electric Vehicle (PHEV)") {
            $9 = "HYBRID"
        }
        if ($9 == "Battery Electric Vehicle (BEV)") {
            $9 = "BATTERY"
        }

        # Print the modified row
        print $0
    }' "$input" > "$output"


Overwriting replace_vehicle_type.sh


In [ ]:
!chmod +x replace_vehicle_type.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Plug-in Hybrid Electric Vehicle (PHEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


Overwriting test_input.csv


In [ ]:
!./replace_vehicle_type.sh test_input.csv test_output.csv


In [ ]:
!cat test_output.csv



VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,BATTERY,Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,BATTERY,Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,HYBRID,Eligibility unknown as battery range has not been researched,200,45000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,HYBRID,Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


MSRP_Zero_clean.sh

In [ ]:
%%writefile replace_msrp_values.sh

#!/bin/bash

input=$1
output=$2

# Step 1: Calculate the median of non-zero MSRP values
awk -F',' '$12 > 0 {print $12}' "$input" | sort -n | awk '{
    a[NR]=$1
} END {
    if (NR % 2 == 1)
        print a[(NR + 1) / 2];
    else
        print (a[NR / 2] + a[NR / 2 + 1]) / 2;
}' > msrp_median.txt

# Read the median value from msrp_median.txt
median=$(<msrp_median.txt)

# Step 2: Calculate the average MSRP for each car make
declare -A make_msrp_sum
declare -A make_msrp_count

# Collect sums and counts for each make (column 7)
awk -F',' '{
    if ($12 > 0) {
        make_msrp_sum[$7] += $12;
        make_msrp_count[$7] += 1;
    }
} END {
    for (make in make_msrp_sum) {
        average_msrp = make_msrp_sum[make] / make_msrp_count[make];
        print make, average_msrp;
    }
}' "$input" > make_msrp_averages.txt

# Step 3: Replace zeros in the Base MSRP column with the average MSRP for that make
# If average MSRP is 0, replace it with the global median

awk -F',' -v OFS=',' -v median="$median" '
    # Load average MSRP for each make into an associative array
    NR == FNR {
        make_avg[$1] = $2;
        next;
    }

    {
        # Get the make of the current row (column 7)
        make = $7;

        # Replace zero MSRP with the average for the make, or with the global median if average is 0
        if ($12 == 0) {
            if (make_avg[make] > 0) {
                $12 = make_avg[make];
            } else {
                $12 = median;
            }
        }
        print $0;
    }
' make_msrp_averages.txt "$input" > "$output"

echo "Zeros in the Base MSRP column have been replaced with the average MSRP for the car make, or with the median if no average exists."


Overwriting replace_msrp_values.sh


In [ ]:
!chmod +x replace_msrp_values.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,0,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,0,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


Overwriting test_input.csv


In [ ]:
!./replace_msrp_values.sh test_input.csv test_output.csv


Zeros in the Base MSRP column have been replaced with the average MSRP for the car make, or with the median if no average exists.


In [ ]:
!cat test_output.csv



VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,200,35000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,35000,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


###remove_fields.sh

In [ ]:
%%writefile remove_columns.sh

#!/bin/bash

input_file=$1
output_file=$2

# Join all column numbers into a space-separated string
columns_to_remove="$3"

cut --complement -d, -f$columns_to_remove "$input_file" > "$output_file"

echo "Columns $columns_to_remove have been removed from $input_file and saved to $output_file"


Overwriting remove_columns.sh


In [ ]:
!chmod +x remove_columns.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Plug-in Hybrid Electric Vehicle (PHEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


Overwriting test_input.csv


In [ ]:
!./remove_columns.sh test_input.csv test_output.csv 3,6


Columns 3,6 have been removed from test_input.csv and saved to test_output.csv


In [ ]:
!cat test_output.csv



VIN,County,State,Postal Code,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,WA,98101,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,WA,98052,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,WA,98402,Chevrolet,Bolt EV,Plug-in Hybrid Electric Vehicle (PHEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,WA,98201,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


###remove_missing_values.sh

In [ ]:
%%writefile remove_missing_rows.sh

#!/bin/bash

input_file=$1
output_file=$2

# Use awk to remove rows with missing values
awk -F',' -v OFS=',' '
{
    # Check if any field is empty (NF - number of fields, $i - field value)
    for (i = 1; i <= NF; i++) {
        if ($i == "" || $i == " ") {
            next  # Skip the row if any field is empty
        }
    }
    # If no empty field, print the row
    print $0
}' "$input_file" > "$output_file"

echo "Rows with missing values have been removed from $input_file and saved to $output_file"


Overwriting remove_missing_rows.sh


In [ ]:
!chmod +x remove_missing_rows.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Plug-in Hybrid Electric Vehicle (PHEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,Everett,WA,,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,140,42000,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


Overwriting test_input.csv


In [ ]:
!./remove_missing_rows.sh test_input.csv test_output.csv


Rows with missing values have been removed from test_input.csv and saved to test_output.csv


In [ ]:
!cat test_output.csv



VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Plug-in Hybrid Electric Vehicle (PHEV),Eligibility unknown as battery range has not been researched,200,45000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10


###replace_zero_range.sh

In [ ]:
%%writefile replace_electric_range.sh

#!/bin/bash

input=$1
output=$2

# Step 1: Calculate the average electric range for each make and model (excluding zero values)
awk -F',' '$11 > 0 {sum[$8 FS $7] += $11; count[$8 FS $7]++}
            END {for (key in sum) print key "," sum[key] / count[key]}' "$input" > averages.txt

# Step 2: Use the calculated averages to replace rows with a zero electric range
awk -F',' -v OFS=',' '
    BEGIN {
        # Load averages into an array
        while ((getline < "averages.txt") > 0) {
            # Split the line into make, model, and average
            split($0, arr, ",")
            make_model[arr[1] FS arr[2]] = arr[3]
        }
    }
    {
        # Check if the electric range is 0 or an empty string (i.e., no value) and replace it with the average
        if ($11 == 0 || length($11) == 0) {
            key = $8 FS $7
            if (key in make_model) {
                $11 = make_model[key]
            }
        }

        # Exclude rows where the electric range is still 0 or an empty string
        if ($11 != 0 && length($11) > 0) {
            print $0
        }
    }' "$input" > "$output"

Overwriting replace_electric_range.sh


In [ ]:
!chmod +x replace_electric_range.sh


In [ ]:
%%writefile test_input.csv

VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
9101,Pierce,Tacoma,WA,98402,2019,Chevrolet,Bolt EV,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not been researched,0,45000,25,3,POINT (122.44 -47.25),Tacoma Power,3.10
1122,Snohomish,Everett,WA,98201,2022,BMW,i3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,0,42000,38,4,POINT (122.20 -47.98),Snohomish County PUD,3.50


Overwriting test_input.csv


In [ ]:
!./replace_electric_range.sh test_input.csv test_output.csv


In [ ]:
!cat test_output.csv


VIN,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
1234,King,Seattle,WA,98101,2020,Tesla,Model S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,320,80000,37,1,POINT (122.33 -47.61),PSE,3.14
5678,King,Redmond,WA,98052,2021,Nissan,Leaf,Battery Electric Vehicle (BEV),Not eligible due to low battery range,150,35000,48,2,POINT (122.12 -47.67),Seattle City Light,3.22
